In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from backprojection.tools import sb1, sb1_r, sb2, wa

In [4]:
import sys

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [6]:
withPlots = 1

In [7]:
fmin = 5.725e9
fMAX = 5.875e9
fc = 5.8e9
B = fMAX - fmin
c = 3e8
kc = 4 * np.pi * fc / c
fs = 10e6

rail = 0
truck = 1
plane = 2
vehicule = rail

if vehicule == rail:
    T = 5e-3
    rampsPerFile = 200
    V = 0.1
if vehicule == truck:
    T = 5e-3
    rampsPerFile = 200
    V = 3
if vehicule == plane:
    T = 600e-6
    rampsPerFile = 1500
    V = 40

alpha = 2 * (fMAX - fmin) / T
samplesPerRamp = int( np.ceil( T * fs ) )
samplesPerFile = rampsPerFile * samplesPerRamp
samplesPerUpRamp = int( samplesPerRamp / 2 )

print("T = {}, samplesPerRamp = {:.2f}, samplesPerFile = {:.2f}".format(T, samplesPerRamp, samplesPerFile))

t1 = np.arange( 0, T, 1/fs)
t2 = np.arange( -T/2, T/2, 1/fs)

T = 0.005, samplesPerRamp = 50000.00, samplesPerFile = 10000000.00


# Build data

In [8]:
nbFiles = 20

In [9]:
a = 0.603354564402 # sinc(a) = 0.5
b = a * 180 / (30 * np.pi) # see the newton notebook for further details about the calculation

In [10]:
hPlane = 90
hCorner = 90
gr_corner = 100
rg_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5
az_corner = V * T * nbFiles * rampsPerFile / 2
noise = 1
print("rg_corner = {:.2f}, gr_corner = {:.2f}, az_corner = {:.2f}".format(rg_corner, gr_corner, az_corner))

rg_corner = 100.00, gr_corner = 100.00, az_corner = 1.00


In [11]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 2, phi = 0.6°


In [35]:
sr_c = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp), dtype=complex )
sr_r = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp) )
for ramp in range( nbFiles * rampsPerFile):
    azn = ramp * T * V
    Rn = ( rg_corner**2 + (azn - az_corner)**2 )**0.5
    noisen = np.random.randn( samplesPerRamp ) * noise / (Rn**2)
    sr_c[ ramp,  0 : samplesPerRamp ] = sb1( t2, Rn, alpha, fc ) * wa( azn - az_corner, rg_corner, b ) \
    + noisen
    sr_r[ ramp,  0 : samplesPerRamp ] = sb1_r( t2, Rn, alpha, fc ) * wa( azn - az_corner, rg_corner, b ) \
    + noisen
    file = ramp/rampsPerFile
    if file == np.floor(file):
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, azn = {:.2f}, wa = {:.1f}, phi = {:.1f}, Rn = {:.1f}".format( 
            file, azn, wa( azn - az_corner, rg_corner ), phi, Rn ) )

0.0, azn = 0.00, wa = 1.0, phi = 0.6, Rn = 100.0
1.0, azn = 0.10, wa = 1.0, phi = 0.5, Rn = 100.0
2.0, azn = 0.20, wa = 1.0, phi = 0.5, Rn = 100.0
3.0, azn = 0.30, wa = 1.0, phi = 0.4, Rn = 100.0
4.0, azn = 0.40, wa = 1.0, phi = 0.3, Rn = 100.0
5.0, azn = 0.50, wa = 1.0, phi = 0.3, Rn = 100.0
6.0, azn = 0.60, wa = 1.0, phi = 0.2, Rn = 100.0
7.0, azn = 0.70, wa = 1.0, phi = 0.2, Rn = 100.0
8.0, azn = 0.80, wa = 1.0, phi = 0.1, Rn = 100.0
9.0, azn = 0.90, wa = 1.0, phi = 0.1, Rn = 100.0
10.0, azn = 1.00, wa = 1.0, phi = 0.0, Rn = 100.0
11.0, azn = 1.10, wa = 1.0, phi = 0.1, Rn = 100.0
12.0, azn = 1.20, wa = 1.0, phi = 0.1, Rn = 100.0
13.0, azn = 1.30, wa = 1.0, phi = 0.2, Rn = 100.0
14.0, azn = 1.40, wa = 1.0, phi = 0.2, Rn = 100.0
15.0, azn = 1.50, wa = 1.0, phi = 0.3, Rn = 100.0
16.0, azn = 1.60, wa = 1.0, phi = 0.3, Rn = 100.0
17.0, azn = 1.70, wa = 1.0, phi = 0.4, Rn = 100.0
18.0, azn = 1.80, wa = 1.0, phi = 0.5, Rn = 100.0
19.0, azn = 1.90, wa = 1.0, phi = 0.5, Rn = 100.0


In [36]:
sr_r0 = sr_r[0,:] * 2
sr_c0 = sr_c[0,:]
sr_r_b = np.real( sr_c ) * 2
sr_r0_b = sr_r_b[0,:]
if withPlots == 1:
    plt.figure()
    plt.plot( np.real( sr_r0 ), 'd-', label="sr_r0" )
    plt.plot( np.real( sr_r0_b ), '.-', label="sr_r0_b" )
    plt.grid()
    plt.legend()

Compare two ways of computing the real signal, by taking the real part of the complex signal $ sr_{rb} = real( sr_c ) * 2 $ or by computing directly in real $ sr_r $.

In [37]:
if withPlots == 1:
    plt.figure()
    plt.subplot(211)
    plt.plot( np.real( np.fft.ifft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.real( np.fft.ifft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.real( np.fft.ifft( sr_r0_b ) ), '.', label="sr_r0_b" )
    plt.grid()
    plt.legend()
    plt.subplot(212)
    plt.plot( np.imag( np.fft.ifft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.imag( np.fft.ifft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.imag( np.fft.ifft( sr_r0_b ) ), '.', label="sr_r0_b" )
    plt.grid()
    plt.legend()

In [39]:
sr_r0[10], sr_c0[10]*2, sr_r0_b[10]

(1.4714890346142544,
 (1.4714890346156573+1.3531980707988078j),
 1.4714890346156573)

In [40]:
sa_r0 = signal.hilbert( sr_r0 ) / 2
if withPlots == 1:
    plt.figure()
    plt.subplot(211)
    plt.plot( np.real( np.fft.fft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.real( np.fft.fft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.real( np.fft.fft( sa_r0 ) ), '.', label="sa_r0" )
    plt.grid()
    plt.legend()
    plt.subplot(212)
    plt.plot( np.imag( np.fft.fft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.imag( np.fft.fft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.imag( np.fft.fft( sa_r0 ) ), '.', label="sa_r0" )
    plt.grid()
    plt.legend()

Compare the complex signal computed using RD_realtocomp and the numpy hilbert complex conjugate.

In [41]:
sa_r0 = np.conjugate( signal.hilbert( sr_r0 ) )[::2]
RDc0 = RD_realtocomp( sr_r0 )
#RDc0 = RDc[0,:] * 2

In [42]:
if withPlots == 1:
    plt.figure()
    ax1 = plt.subplot(221)
    plt.plot( np.real( sa_r0 ), 'd-', label="sa_r0" )
    plt.plot( np.real( RDc0 ), '.', label="RDc" )
    plt.plot( np.real( sr_c0[::2]*2 ), 'xy', label="sr_c0[::2]" )
    plt.grid()
    plt.legend()
    plt.subplot(222, sharex=ax1)
    plt.plot( np.imag( sa_r0 ), 'd-', label="sa_r0" )
    plt.plot( np.imag( RDc0 ), '.', label="RDc" )
    plt.plot( np.imag( sr_c0[::2]*2 ), 'xy', label="sr_c0[::2]" )
    plt.grid()
    plt.legend()
    ax2 = plt.subplot(223)
    plt.plot( np.real( np.fft.ifft( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.real( np.fft.ifft( RDc0 ) ), '.', label="RDc" )
    plt.plot( np.real( np.fft.ifft( sr_c0[::2]*2 ) ), 'xy', label="sr_c0[::2]" )
    plt.grid()
    plt.legend()
    plt.subplot(224, sharex=ax2)
    plt.plot( np.imag( np.fft.ifft( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.imag( np.fft.ifft( RDc0 ) ), '.', label="RDc" )
    plt.plot( np.imag( np.fft.ifft( sr_c0[::2]*2 ) ), 'xy', label="sr_c0[::2]" )
    plt.grid()

Compare the analytic signal computed with numpy hilbert complex conjugate and the complex signal.

**Conclusion:** little differences at the beginning of the series.

In [43]:
sa_r0_b = np.conjugate( signal.hilbert( sr_r0 ) )

In [45]:
if withPlots == 1:
    plt.figure()
    ax1 = plt.subplot(221)
    plt.plot( np.real( sa_r0_b ), 'd-', label="sa_r0_b" )
    plt.plot( np.real( sr_c0*2 ), 'xy', label="sr_c0" )
    plt.grid()
    plt.legend()
    plt.subplot(222, sharex=ax1)
    plt.plot( np.imag( sa_r0_b ), 'd-', label="sa_r0_b" )
    plt.plot( np.imag( sr_c0*2 ), 'xy', label="sr_c0" )
    plt.grid()
    plt.legend()
    ax2 = plt.subplot(223)
    plt.plot( np.real( np.fft.ifft( sa_r0_b ) ), 'd-', label="sa_r0_b" )
    plt.plot( np.real( np.fft.ifft( sr_c0*2 ) ), '.', label="RDc" )
    plt.grid()
    plt.legend()
    plt.subplot(224, sharex=ax2)
    plt.plot( np.imag( np.fft.ifft( sa_r0_b ) ), 'd-', label="sa_r0_b" )
    plt.plot( np.imag( np.fft.ifft( sr_c0*2 ) ), '.', label="RDc" )
    plt.grid()

In [ ]:
sr_a = np.conjugate( signal.hilbert( sr_r, axis=1 ) )[:,::2]

In [ ]:
srf = np.fft.ifft( sr_a, axis=1 )

In [ ]:
sr_r.shape, sr_a.shape, srf.shape

In [ ]:
srf_dB = 20 * np.log10( np.abs( srf ) )
srf_min = np.amin(srf_dB)
srf_max = np.amax(srf_dB)
srf_med = np.median(srf_dB)
print("srf_min = {:.2f}, srf_max = {:.2f}, srf_med = {:.2f}".format(srf_min, srf_max, srf_med))

In [ ]:
if withPlots == 1:
    plt.figure()
    plt.imshow( srf_dB, cmap='jet' )

In [ ]:
srf[1000, 1000]

In [ ]:
RDc = np.zeros( (sr_r.shape[0], int(sr_r.shape[1]/2) ), dtype=complex )
for k in range( sr_r.shape[0] ):
    RDc[k, :] = RD_realtocomp( sr_r[k, :] )

In [ ]:
ifft_RDc_dB = 20 * np.log10( np.abs( np.fft.ifft( RDc, axis=1 ) ) )

In [ ]:
if withPlots == 1:
    plt.figure()
    plt.imshow( ifft_RDc_dB, cmap='jet' )

# Save data

Keep only up ramps to build the complex incoming signal with numpy hilbert complex conjugate.

In [47]:
complexSignal = np.conjugate( signal.hilbert( sr_r[:, 0:samplesPerUpRamp], axis=1 )[:, ::2] )

srf.shape = (4000, 12500)


In [ ]:
srf = np.fft.ifft( complexSignal, axis=1 )
print( "srf.shape = {}".format(srf.shape))

In [48]:
srf_dB = 20 * np.log10( np.abs( srf ) )

In [49]:
if withPlots == 1:
    plt.figure()
    plt.imshow( srf_dB, cmap='jet' )

In [51]:
data_dir = "/home/pleroy/DATA/SIMU"
np.save( data_dir + "/complexSignal_{}_{}".format(0, nbFiles-1), complexSignal )

# Time series

In [ ]:
plt.figure()
r = 100.5
tau = 2 * r / c
print( "tau = {}, f = {}".format(tau, alpha*tau) )
plt.subplot( 211 )
plt.plot( np.real( sb1( t1[0:3000], r ) ), '.' )
plt.plot( np.real( sb2( t1[0:3000], r ) ) )
ax = plt.gca()
plt.subplot( 212, sharex=ax )
plt.plot( np.imag( sb1( t1[0:3000], r ) ) )
plt.plot( np.imag( sb2( t1[0:3000], r ) ) )
ax = plt.gca()
ax.set_xlim(0, 100)

In [ ]:
plt.figure()
r = 1000.5
tau = 2 * r / c
print( "tau = {}, f = {}".format(tau, alpha*tau) )
sb1_t1 = np.real( sb1( t1[0:3000], r ) )
sb2_t1 = np.real( sb2( t1[0:3000], r ) )
plt.subplot(311)
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb1_t1 ) ) ) )
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb2_t1 ) ) ) )
plt.subplot(312)
plt.plot( np.real(np.fft.ifft( sb1_t1 ) ) )
plt.plot( np.real(np.fft.ifft( sb2_t1 ) ) )
plt.subplot(313)
plt.plot( np.imag(np.fft.ifft( sb1_t1 ) ) )
plt.plot( np.imag(np.fft.ifft( sb2_t1 ) ) )